# Time Analysis
* In this notebook, we look at the translation times DeepL and GPT4.1 had for the same tasks

In [6]:
from os.path import join
import pandas as pd
import json
import os

os.makedirs('time_results', exist_ok=True)

parts = ['ep-gpt', 'ep-deepl', 'flores-gpt', 'flores-deepl', 'opus-gpt', 'opus-deepl']

with open(join('translations', 'info.json'), 'r') as f:
    prefix2file = json.load(f)

for part in parts:
    data = {'Label':[], 'Time':[]}
    for prefix, info in prefix2file.items():
        if prefix.startswith(part):
            src_lang, tgt_lang = prefix.split('-')[2], prefix.split('-')[3]
            duration = info['log']['end'] - info['log']['start']
            data['Label'].append(f'{src_lang}-{tgt_lang}')
            data['Time'].append(duration)
    df = pd.DataFrame(data)
    df.to_csv(join('time_results', f'{part}.csv'))

In [7]:
from scripts.scoring import create_matrix_from_csv
import os
file2df = {f.replace('.csv', ''):{'file':join('time_results', f), 'df':None} for f in os.listdir('time_results')}

for f, content in file2df.items():
    file_path = content['file']
    df = create_matrix_from_csv(file_path, metric='Time')
    file2df[f]['df'] = df


In [8]:
file2df['ep-deepl']['df'].round(1)

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,11.2,16.4,11.4,16.4,16.4,16.8,11.1,16.4,16.2,11.3
de,16.2,NaN,11.3,6.1,21.5,11.1,11.1,6.1,11.1,6.0,11.2
el,16.4,11.3,NaN,11.0,11.2,11.2,11.3,11.3,16.4,11.3,16.3
en,11.0,5.8,13.6,NaN,5.8,5.8,6.0,11.0,21.8,11.0,11.0
es,11.0,11.2,11.2,5.9,NaN,11.4,11.1,11.1,11.1,11.1,16.1
fi,16.2,11.1,16.3,10.9,11.0,NaN,11.0,11.0,11.2,11.2,11.0
fr,16.2,11.1,16.4,5.9,11.1,16.2,NaN,16.2,16.4,11.2,11.0
it,16.2,16.1,16.4,11.4,11.2,11.0,16.2,NaN,11.1,16.1,16.2
nl,16.2,11.0,11.3,5.8,11.0,11.0,16.1,16.2,NaN,11.1,16.1
pt,16.1,16.2,16.2,21.1,16.4,16.2,16.2,21.3,16.1,NaN,16.1


In [9]:
file2df['ep-gpt']['df'].round(1)

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,157.0,279.0,142.0,218.0,288.0,167.0,232.0,245.0,214.0,202.0
de,178.0,NaN,269.0,252.0,202.0,205.0,228.0,173.0,209.0,211.0,192.0
el,281.0,231.0,NaN,240.0,221.0,257.0,220.0,279.0,171.0,237.0,201.0
en,155.0,194.0,239.0,NaN,159.0,275.0,176.0,177.0,207.0,121.0,105.0
es,244.0,194.0,255.0,116.0,NaN,245.0,242.0,159.0,215.0,146.0,216.0
fi,243.0,241.0,263.3,139.0,220.0,NaN,206.0,233.0,301.0,200.0,270.0
fr,244.0,272.0,291.0,145.0,162.0,273.0,NaN,222.0,289.0,224.0,290.0
it,267.0,193.0,301.0,153.0,182.0,286.0,186.0,NaN,266.0,211.0,299.0
nl,263.0,234.0,218.0,190.0,215.0,290.0,264.0,268.0,NaN,203.0,293.0
pt,241.0,271.0,289.0,89.0,217.0,291.0,208.0,221.0,224.0,NaN,235.0


In [10]:
file2df['flores-deepl']['df'].round(1)

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,11.1,11.2,10.9,5.9,11.0,5.9,5.9,11.1,11.2,11.5
de,11.1,NaN,11.1,6.0,6.1,11.1,6.0,5.9,12.6,6.0,11.0
el,16.2,11.1,NaN,11.0,11.0,11.1,11.0,16.2,16.2,11.1,16.2
en,10.8,5.9,11.0,NaN,6.2,10.9,5.8,5.8,10.8,10.8,10.8
es,16.1,11.0,11.3,5.8,NaN,11.2,11.1,11.0,16.1,11.0,16.1
fi,16.1,11.0,11.2,11.3,11.1,NaN,11.0,11.1,11.0,11.1,16.3
fr,11.1,11.1,16.4,5.8,11.3,11.1,NaN,16.2,11.3,11.1,11.2
it,11.1,11.1,16.2,5.8,11.2,16.4,11.1,NaN,16.4,11.1,11.1
nl,6.0,11.1,16.4,5.8,6.0,16.2,11.0,11.0,NaN,11.0,11.2
pt,16.2,11.1,11.2,5.8,11.0,11.1,11.1,11.1,11.0,NaN,16.2


In [11]:
file2df['flores-gpt']['df'].round(1)

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,179.8,207.8,129.0,142.8,295.4,170.7,105.2,117.0,169.3,109.1
de,144.3,NaN,183.2,108.0,164.8,208.5,280.6,219.3,119.6,159.8,137.1
el,263.1,243.4,NaN,220.0,156.0,288.8,203.1,189.3,171.1,154.1,160.4
en,187.0,170.0,299.0,NaN,135.0,223.0,137.0,185.0,149.0,188.0,134.0
es,207.1,162.9,264.9,103.0,NaN,216.4,203.8,167.1,156.0,120.2,192.0
fi,195.9,172.2,259.5,157.0,153.1,NaN,197.4,230.4,162.2,212.3,163.3
fr,130.2,284.8,263.8,122.0,151.5,201.1,NaN,138.6,208.8,120.0,217.8
it,160.4,215.5,274.8,130.0,118.2,201.9,110.1,NaN,137.1,155.2,196.8
nl,179.3,145.8,296.0,106.0,99.7,233.5,194.0,160.2,NaN,172.4,169.1
pt,244.8,256.0,195.5,170.0,92.9,251.7,180.1,138.3,150.9,NaN,185.7


In [12]:
file2df['opus-deepl']['df'].round(1)

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,NaN,NaN,10.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
de,NaN,NaN,NaN,6.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
el,NaN,NaN,NaN,6.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
en,10.9,5.7,5.7,NaN,5.7,5.7,6.8,7.4,5.7,5.7,7.8
es,NaN,NaN,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fi,NaN,NaN,NaN,5.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fr,NaN,NaN,NaN,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
it,NaN,NaN,NaN,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nl,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pt,NaN,NaN,NaN,5.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
file2df['opus-gpt']['df'].round(1)

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,NaN,NaN,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
de,NaN,NaN,NaN,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
el,NaN,NaN,NaN,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
en,137.0,114.0,217.0,NaN,94.0,179.0,182.0,90.0,137.0,106.0,83.0
es,NaN,NaN,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fi,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fr,NaN,NaN,NaN,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
it,NaN,NaN,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nl,NaN,NaN,NaN,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pt,NaN,NaN,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Statistics
* To compare this data more efficiently, it makes sense to employ techniques such as normalization

In [14]:
norm = {}
for f, content in file2df.items():
    df = content['df']
    normalized_df = (df - df.mean().mean()) / df.stack().std()
    norm[f] = normalized_df

In [16]:
norm['ep-deepl'].round(1)

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,-0.5,1.0,-0.4,1.0,1.0,1.1,-0.5,1.0,0.9,-0.4
de,0.9,NaN,-0.5,-1.9,2.4,-0.5,-0.5,-1.9,-0.5,-1.9,-0.5
el,1.0,-0.5,NaN,-0.5,-0.5,-0.5,-0.5,-0.5,1.0,-0.4,0.9
en,-0.5,-2.0,0.2,NaN,-2.0,-2.0,-1.9,-0.5,2.5,-0.5,-0.5
es,-0.5,-0.5,-0.5,-1.9,NaN,-0.4,-0.5,-0.5,-0.5,-0.5,0.9
fi,0.9,-0.5,0.9,-0.6,-0.5,NaN,-0.5,-0.5,-0.5,-0.5,-0.5
fr,0.9,-0.5,1.0,-2.0,-0.5,0.9,NaN,0.9,0.9,-0.5,-0.5
it,0.9,0.9,1.0,-0.4,-0.5,-0.5,0.9,NaN,-0.5,0.9,0.9
nl,0.9,-0.5,-0.5,-2.0,-0.5,-0.5,0.9,0.9,NaN,-0.5,0.9
pt,0.9,0.9,0.9,2.3,1.0,0.9,0.9,2.3,0.9,NaN,0.9


In [17]:
norm['ep-gpt'].round(1)

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,-1.3,1.2,-1.6,-0.1,1.4,-1.1,0.2,0.5,-0.1,-0.4
de,-0.9,NaN,1.0,0.6,-0.4,-0.3,0.1,-1.0,-0.2,-0.2,-0.6
el,1.2,0.2,NaN,0.4,0.0,0.7,-0.0,1.2,-1.0,0.3,-0.4
en,-1.3,-0.5,0.4,NaN,-1.2,1.1,-0.9,-0.9,-0.3,-2.0,-2.3
es,0.5,-0.5,0.7,-2.1,NaN,0.5,0.4,-1.2,-0.1,-1.5,-0.1
fi,0.4,0.4,0.9,-1.7,-0.0,NaN,-0.3,0.2,1.6,-0.4,1.0
fr,0.5,1.0,1.4,-1.5,-1.2,1.1,NaN,0.0,1.4,0.1,1.4
it,0.9,-0.6,1.6,-1.4,-0.8,1.3,-0.7,NaN,0.9,-0.2,1.6
nl,0.9,0.3,-0.1,-0.6,-0.1,1.4,0.9,1.0,NaN,-0.4,1.5
pt,0.4,1.0,1.4,-2.7,-0.1,1.4,-0.3,0.0,0.1,NaN,0.3


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

parts = ['ep', 'flores', 'opus']

for part in parts:
    gpt = f'{part}-gpt'
    deepl = f'{part}-deepl'
    gpt_df = file2df[gpt]['df']
    deepl_df = file2df[deepl]['df']

    gpt_flat = gpt_df.values.flatten()
    deepl_flat = deepl_df.values.flatten()
    
    mask = ~np.isnan(gpt_flat) & ~np.isnan(deepl_flat)

    # Compute Pearson correlation
    corr, pval = pearsonr(gpt_flat[mask], deepl_flat[mask])
    print(f"Pearson correlation: {corr:.2f}")
    print(f"p-value: {pval:3f}")

Pearson correlation: 0.19
p-value: 0.048976
Pearson correlation: 0.28
p-value: 0.003265
Pearson correlation: -0.26
p-value: 0.268835
